In [1]:
#use xarray package to create new netcdfs for pattern correlation
#compare PNW 2021 Heatwave to model data to find pattern corr. Lowered resolution of ERA5 to allow comparison to model.

#generate excel sheet of pattern correlations for each EC earth model day against PNW 2021 heatwave
scenario = 'ssp126'

##IMPORT PACKAGES
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import cartopy.feature as cfeature ## for lakes/borders etc
countries_50m = cfeature.NaturalEarthFeature('cultural','admin_0_countries','50m',edgecolor='k',facecolor='none')
import cartopy.crs as ccrs
import numpy as np
from netCDF4 import Dataset  
import xarray as xr
import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import os
import iris
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units
import iris.coord_categorisation
import iris.plot as iplt
import iris.quickplot as qplt
import iris.analysis.stats
import pandas as pd
import seaborn as sns
import glob
import datetime
import cf_units as unit

crs = ccrs.PlateCarree()

import warnings
warnings.filterwarnings("ignore")

/apps/developers/compilers/canopy3/2.1.3/1/bit-64/Canopy/edm/envs/User/lib/python3.5/site-packages/matplotlib/ticker.py:1693: UserWarning: Steps argument should be a sequence of numbers
increasing from 1 to 10, inclusive. Behavior with
values outside this range is undefined, and will
raise a ValueError in future versions of mpl.
  warnings.warn('Steps argument should be a sequence of numbers\n'


In [43]:
#read in file of maxtemps exceeding historical 99.5th pctl
df_PNW126 = pd.read_excel('df_PNW126_final.xlsx', index_col=0)  
#create lists
date_string_list=[]
member_list=[]

dates = pd.to_datetime(df_PNW126['Date'], format='%Y%m%d')
dates.dt.strftime('%Y-%m-%d')

for date in dates:
    date_str= str(date)
    date_short = date_str[0:10]
    date_string_list.append(date_short)
    
members = df_PNW126['Member'].tolist()

for member in members:
    member_short = member[0:10]
    member_list.append(member_short)
    
print(len(member_list))
print(len(date_string_list))

In [12]:
#date_string_list = ['2094-07-18','2094-07-28','2032-07-24','2044-07-23','2079-07-24','2079-07-23','2068-07-29','2099-07-11','2024-07-24','2061-07-31','2082-07-18','2064-07-09','2054-07-26','2090-07-26','2097-07-31','2038-07-30','2094-07-23','2059-07-23','2050-07-31','2050-07-02','2076-07-29','2097-07-26','2095-07-25','2088-07-09','2086-07-12','2073-07-27','2068-07-21','2028-07-31','2061-07-21','2020-07-22','2056-07-28','2050-07-07','2096-07-31','2070-07-29','2073-07-23','2073-07-18','2034-07-13','2069-07-31','2044-07-30','2023-07-12','2051-07-19','2045-07-25','2066-07-13','2091-07-20','2033-07-17','2100-07-27','2062-07-27','2086-07-27','2096-06-23','2049-07-25','2056-07-26','2053-07-30','2060-07-27','2043-07-30','2042-07-13','2046-07-19','2095-07-03','2035-07-31','2028-07-03','2052-07-28','2041-07-27','2052-07-31','2084-07-26','2095-07-30','2037-07-28','2067-07-10','2035-07-07','2074-07-31','2085-07-21','2070-07-20','2088-06-30','2088-07-01','2019-07-07','2032-07-30','2069-07-08','2068-07-10','2041-07-23','2070-07-10','2095-07-26','2075-07-30','2042-07-11','2068-07-20','2068-07-19','2093-07-22','2094-07-07','2089-07-31','2060-07-07','2060-07-23','2061-07-22','2074-07-08','2060-07-26','2063-07-23','2064-07-31']
#member_list = ['r110i1p1f1','r111i1p1f1','r101i1p1f1','r106i1p1f1','r109i1p1f1','r102i1p1f1','r102i1p1f1','r102i1p1f1','r103i1p1f1','r102i1p1f1','r110i1p1f1','r106i1p1f1','r103i1p1f1','r101i1p1f1','r106i1p1f1','r106i1p1f1','r106i1p1f1','r101i1p1f1','r112i1p1f1','r104i1p1f1','r111i1p1f1','r103i1p1f1','r105i1p1f1','r106i1p1f1','r110i1p1f1','r101i1p1f1','r103i1p1f1','r111i1p1f1','r109i1p1f1','r112i1p1f1','r102i1p1f1','r108i1p1f1','r109i1p1f1','r107i1p1f1','r105i1p1f1','r103i1p1f1','r103i1p1f1','r107i1p1f1','r104i1p1f1','r107i1p1f1','r106i1p1f1','r111i1p1f1','r109i1p1f1','r109i1p1f1','r109i1p1f1','r109i1p1f1','r110i1p1f1','r108i1p1f1','r107i1p1f1','r106i1p1f1','r110i1p1f1','r109i1p1f1','r104i1p1f1','r105i1p1f1','r103i1p1f1','r108i1p1f1','r107i1p1f1','r111i1p1f1','r106i1p1f1','r111i1p1f1','r102i1p1f1','r101i1p1f1','r106i1p1f1','r101i1p1f1','r110i1p1f1','r103i1p1f1','r110i1p1f1','r109i1p1f1','r110i1p1f1','r104i1p1f1','r109i1p1f1','r109i1p1f1','r104i1p1f1','r111i1p1f1','r105i1p1f1','r101i1p1f1','r104i1p1f1','r109i1p1f1','r103i1p1f1','r103i1p1f1','r106i1p1f1','r106i1p1f1','r108i1p1f1','r101i1p1f1','r105i1p1f1','r111i1p1f1','r103i1p1f1','r108i1p1f1','r106i1p1f1','r110i1p1f1','r107i1p1f1','r111i1p1f1','r103i1p1f1']

In [45]:
def sorted(cube, coord):
    coord_to_sort = cube.coord(coord)
    assert coord_to_sort.ndim == 1, 'One dim coords only please.'
    dim, = cube.coord_dims(coord_to_sort)
    index = [slice(None)] * cube.ndim
    index[dim] = np.argsort(coord_to_sort.points)
    return cube[tuple(index)]

In [46]:
#inputs
#membername = 'r101i1p1f1'

#Lat long boundaries
#Specify min and max values of latitude and longitude 
lat_min = 30
lat_max =70
lon_min = 215 #-145
lon_max = 290 #-70

path = '/nfs/annie/earamay/Prash/'+scenario+'/'
fn2 = '/nfs/annie/ee21pnm/210628_pcorr_small.nc' 
set_date = dt.date(2021,6,28)


output_name = "/nfs/see-fs-01_teaching/ee21pnm/Spreadsheets/PNW_Corr_"+scenario+"_2021_06_28_v3.xlsx"

#save histograms
pathout ="/nfs/see-fs-01_teaching/ee21pnm/Project/output/Pattern_Corr_Histograms/PNW"

In [47]:
#load control day cube (eg 28 June 2021)
cube1 = iris.load_cube(fn2)

longitude = cube1.coord('longitude')
latitude = cube1.coord('latitude')


In [48]:
#START PATTERN CORR
p_corr_list = []

count = 0
for date_string in date_string_list:
    #date_text = custom_strftime('{S} %B %Y', dt.datetime.strptime(date_string, '%Y-%m-%d'))
    year = int(date_string[0:4])
    model_month = int(date_string[5:7])
    model_day = int(date_string[8:10])

    #load model day

    #date
    model_day2 = dt.datetime(year, model_month, model_day,12)
    
    #specify member
    membername = member_list[count]

    ##Read in geopotential data
    zfile = glob.glob(path + "Z/zg_day_EC-Earth3_"+scenario+"_" + membername + "_gr_" +str(year)+ "0101-" +str(year)+ "1231.nc")
    fn = zfile[0]
    cube2 = iris.load_cube(fn)

    #longitude2 = cube2.coord('longitude')
    #latitude2 = cube2.coord('latitude')
    #pressure2 = cube2.coord('air_pressure')


    # Functions to identify latitudes and longitudes we want to subset to
    def region_lat(input):
        return lat_min  <= input <= lat_max 

    def region_long(input):
        return lon_min  <= input <= lon_max 

    def pressure_500(input):
        return input == 50000

    region_con = iris.Constraint(latitude = region_lat, longitude = region_long)
    pressure_con = iris.Constraint( air_pressure= pressure_500)
    date_con = iris.Constraint(time=lambda t: t.point == unit.date2num(model_day2, 'days since 1850-01-01 00:00:00','proleptic_gregorian'))


    # Subset the cube to the location of interest
    cube2_subset = cube2.extract(region_con & pressure_con & date_con)
    #find spatial average for each day
    coords = ('longitude', 'latitude')

    #sort cube by increasing latitude
    cube1_sorted = sorted(cube1, 'latitude')

    #apply weighting
    cosinelat_weights_large=iris.analysis.cartography.cosine_latitude_weights(cube1_sorted)
    cosinelat_weights_small=iris.analysis.cartography.cosine_latitude_weights(cube2_subset)
    
    longitude2 = cube2_subset.coord('longitude')
    latitude2 = cube2_subset.coord('latitude')

    latitude1b = cube1_sorted.coord('latitude')
    longitude1b = cube1_sorted.coord('longitude')

    #resample cube1 to make it smaller
    sample_points = [('longitude', longitude2.points),
                     ('latitude',  latitude2.points)]
    resampled_cube1 = cube1_sorted.interpolate(sample_points, iris.analysis.Linear())

    #create duplicate cube in style of cube1 so the correlation will work
    duplicate_cube2 = resampled_cube1.copy()
    duplicate_cube2.data = cube2_subset.data

    #replace its data with cube2 data
    duplicate_cube2.data = cube2_subset.data
    cube2_final = duplicate_cube2

    #convert units
    cube1_final = (resampled_cube1 /9.81)

    #calculate correlation with model data
    correlation = iris.analysis.stats.pearsonr(cube1_final, cube2_final, weights=cosinelat_weights_small)

    print(correlation.data)
    p_corr_value = correlation.data

    p_corr_list.append(p_corr_value) 
    count = count + 1




0.789026030640765
0.7874032359392152
0.885686750072948
0.7603301624774036
0.7232872351010942
0.9281575358763927
0.7858696873962107
0.8027103190631092
0.7833346854270787
0.8462342753434784
0.8566259077178328
0.7946142846666364
0.8060366461878423
0.8681820865593641
0.7807679317893507
0.8322948208954745
0.8057809757377531
0.7847419837956956
0.8259423916872148
0.8686090166350575
0.8799938244602911
0.6854881140040787
0.9107273798112062
0.8580724423588806
0.8388014793247297
0.7851285430227658
0.8275911583410535
0.8529430561390192
0.6262885410747683
0.8093976034514065
0.8202097801095334
0.6715154080326365
0.759569973709607
0.868404994057967
0.7913253250452524
0.7630919958046374
0.8219477761403094
0.6428515989905864
0.8866839544599481
0.7459277272711371
0.698344047493526
0.6549142379273776
0.8916054550677884
0.8905354386011214
0.8248789465837211
0.8554754568995158
0.8777039539923729
0.8583466221913701
0.8192931426246989
0.8207673386617454
0.7044133729228932
0.7412923105243008
0.803790203816401

In [49]:
#make dataframe
#after all dates are done
model_series = pd.DataFrame({'dates':date_string_list,'member': member_list,'p_corr': p_corr_list}, columns=['dates','member','p_corr'])
model_series

,dates,member,p_corr
0,2020-06-27,r103i1p1f1,0.789026030640765
1,2020-06-27,r108i1p1f1,0.7874032359392152
2,2029-06-29,r104i1p1f1,0.885686750072948
3,2030-06-27,r111i1p1f1,0.7603301624774036
4,2033-06-14,r101i1p1f1,0.7232872351010942
5,2033-06-28,r108i1p1f1,0.9281575358763927
6,2033-06-20,r109i1p1f1,0.7858696873962107
7,2037-06-16,r101i1p1f1,0.8027103190631092
8,2038-06-27,r104i1p1f1,0.7833346854270787
9,2038-06-30,r106i1p1f1,0.8462342753434784


In [50]:
model_series.to_excel(output_name)